In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.dates as mtd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import confusion_matrix, classification_report, plot_roc_curve,accuracy_score
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier  
from sklearn.linear_model import LogisticRegression 
from collections import Counter
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from numpy import mean
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
flightsampledata = pd.read_csv('/content/drive/MyDrive/AbiML/Internship Tasks/internshipDatas/Systematic_sampling_10%_Flight_dataset1_new.csv')
flightsampledata.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,MKT_UNIQUE_CARRIER,BRANDED_CODE_SHARE,MKT_CARRIER_AIRLINE_ID,MKT_CARRIER,MKT_CARRIER_FL_NUM,SCH_OP_UNIQUE_CARRIER,SCH_OP_CARRIER_AIRLINE_ID,SCH_OP_CARRIER,SCH_OP_CARRIER_FL_NUM,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_FIPS,ORIGIN_STATE_NM,ORIGIN_WAC,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_FIPS,DEST_STATE_NM,DEST_WAC,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,DEP_DEL15,DEP_DELAY_GROUP,DEP_TIME_BLK,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,DIVERTED,DUP,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,TOTAL_ADD_GTIME,LONGEST_ADD_GTIME
0,2021,1,1,17,7,2021-01-17,DL,DL,19790,DL,1089,NaN,NaN,NaN,NaN,DL,19790,DL,N326NB,1089,11298,1129806,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,1153,1151.0,-2.0,0.0,0.0,-1.0,1100-1159,28.0,1219.0,1525.0,11.0,1523,1536.0,13.0,13.0,0.0,0.0,1500-1559,0.0,NaN,0.0,N,150.0,165.0,126.0,1.0,986.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,1,1,17,7,2021-01-17,DL,DL,19790,DL,1097,NaN,NaN,NaN,NaN,DL,19790,DL,N356NW,1097,14869,1486903,34614,SLC,"Salt Lake City, UT",UT,49,Utah,87,11298,1129806,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,948,939.0,-9.0,0.0,0.0,-1.0,0900-0959,31.0,1010.0,1312.0,15.0,1333,1327.0,-6.0,0.0,0.0,-1.0,1300-1359,0.0,NaN,0.0,N,165.0,168.0,122.0,1.0,989.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,1,1,17,7,2021-01-17,DL,DL,19790,DL,1112,NaN,NaN,NaN,NaN,DL,19790,DL,N878DN,1112,14747,1474703,30559,SEA,"Seattle, WA",WA,53,Washington,93,14869,1486903,34614,SLC,"Salt Lake City, UT",UT,49,Utah,87,1010,1006.0,-4.0,0.0,0.0,-1.0,1000-1059,16.0,1022.0,1242.0,8.0,1314,1250.0,-24.0,0.0,0.0,-2.0,1300-1359,0.0,NaN,0.0,N,124.0,104.0,80.0,1.0,689.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,1,1,17,7,2021-01-17,DL,DL,19790,DL,1126,NaN,NaN,NaN,NaN,DL,19790,DL,N3732J,1126,10397,1039707,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,14730,1473004,33044,SDF,"Louisville, KY",KY,21,Kentucky,52,1313,1310.0,-3.0,0.0,0.0,-1.0,1300-1359,14.0,1324.0,1418.0,6.0,1426,1424.0,-2.0,0.0,0.0,-1.0,1400-1459,0.0,NaN,0.0,N,73.0,74.0,54.0,1.0,321.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,1,1,17,7,2021-01-17,DL,DL,19790,DL,1140,NaN,NaN,NaN,NaN,DL,19790,DL,N819DN,1140,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,10397,1039707,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,605,559.0,-6.0,0.0,0.0,-1.0,0600-0659,26.0,625.0,755.0,3.0,823,758.0,-25.0,0.0,0.0,-2.0,0800-0859,0.0,NaN,0.0,N,138.0,119.0,90.0,1.0,594.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
flightsampledata.dropna(subset=['DEP_DEL15','TAXI_OUT','AIR_TIME'],inplace=True)

In [ ]:
flightsampledata.isna().sum()

YEAR                        0
QUARTER                     0
MONTH                       0
DAY_OF_MONTH                0
DAY_OF_WEEK                 0
                        ...  
SECURITY_DELAY         189874
LATE_AIRCRAFT_DELAY    189874
FIRST_DEP_TIME         213921
TOTAL_ADD_GTIME        213921
LONGEST_ADD_GTIME      213921
Length: 74, dtype: int64

In [ ]:
systematic_sample_selectedcols=flightsampledata[['YEAR','MONTH','DAY_OF_MONTH','DAY_OF_WEEK','FL_DATE','ORIGIN','ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','TAIL_NUM','AIR_TIME','OP_CARRIER_FL_NUM','DISTANCE','OP_CARRIER_AIRLINE_ID','DEP_DEL15','CRS_DEP_TIME','CRS_ELAPSED_TIME','TAXI_OUT','DEP_TIME','ARR_TIME]]

In [ ]:
from datetime import date
systematic_sample_selectedcols['FL_DATE']=pd.to_datetime(systematic_sample_selectedcols['FL_DATE'],format='%Y-%m-%d')

In [ ]:
#systematic_sample_selectedcols['ORIGIN_AIRPORT_ID']=systematic_sample_selectedcols['DAY_OF_MONTH'].astype(int)
#systematic_sample_selectedcols['DEST_AIRPORT_ID']=systematic_sample_selectedcols['MONTH'].astype(int)
#systematic_sample_selectedcols['YEAR']=systematic_sample_selectedcols['YEAR'].astype(int)
systematic_sample_selectedcols.dtypes

NameError: ignored

## Creating new column- WEEK_OF_MONTH

In [ ]:
import datetime
import calendar

def week_of_month(tgtdate):
    #tgtdate = tgtdate.to_datetime()

    days_this_month = calendar.mdays[tgtdate.month]
    for i in range(1, days_this_month):
        d = datetime.datetime(tgtdate.year, tgtdate.month, i)
        if d.day - d.weekday() > 0:
            startdate = d
            break
    # now we can use the modulo 7 appraoch
    return (tgtdate - startdate).days //7 + 2

systematic_sample_selectedcols['WEEK_OF_MONTH'] = systematic_sample_selectedcols['FL_DATE'].apply(week_of_month)
systematic_sample_selectedcols[['FL_DATE','WEEK_OF_MONTH']].sort_values('WEEK_OF_MONTH',ascending=False) .head()

,FL_DATE,WEEK_OF_MONTH
73763,2021-03-29,6
190172,2021-05-31,6
86852,2021-03-30,6
86851,2021-03-30,6
86850,2021-03-30,6


In [ ]:
systematic_sample_selectedcols['WEEK_OF_MONTH'].value_counts()

4    49502
3    49402
2    48483
5    47621
1    13755
6     6366
Name: WEEK_OF_MONTH, dtype: int64

## Creating New column--> Week days==0, Weekends == 1(DAY_OF_WEEK_days_end)

In [ ]:
#'DAY_OF_WEEK' because delay could depend on traffic in specific days: classify it to weekend or working day
def get_day_category(day_of_week):
    if day_of_week <= 5:
        return 0 #'working day'
    elif day_of_week > 5:
        return 1 #'weekend'
systematic_sample_selectedcols['DAY_OF_WEEK_days_end'] = systematic_sample_selectedcols['DAY_OF_WEEK']
   
systematic_sample_selectedcols['DAY_OF_WEEK_days_end'] = systematic_sample_selectedcols.DAY_OF_WEEK_days_end.apply(get_day_category)
systematic_sample_selectedcols.head()

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,ORIGIN,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,TAIL_NUM,AIR_TIME,OP_CARRIER_FL_NUM,DISTANCE,OP_CARRIER_AIRLINE_ID,DEP_DEL15,CRS_DEP_TIME,CRS_ELAPSED_TIME,TAXI_OUT,DEP_TIME,WEEK_OF_MONTH,DAY_OF_WEEK_days_end
0,2021,1,17,7,2021-01-17,DFW,11298,11433,N326NB,126.0,1089,986.0,19790,0.0,1153,150.0,28.0,1151.0,3,1
1,2021,1,17,7,2021-01-17,SLC,14869,11298,N356NW,122.0,1097,989.0,19790,0.0,948,165.0,31.0,939.0,3,1
2,2021,1,17,7,2021-01-17,SEA,14747,14869,N878DN,80.0,1112,689.0,19790,0.0,1010,124.0,16.0,1006.0,3,1
3,2021,1,17,7,2021-01-17,ATL,10397,14730,N3732J,54.0,1126,321.0,19790,0.0,1313,73.0,14.0,1310.0,3,1
4,2021,1,17,7,2021-01-17,DTW,11433,10397,N819DN,90.0,1140,594.0,19790,0.0,605,138.0,26.0,559.0,3,1


In [ ]:
systematic_sample_selectedcols['DAY_OF_WEEK_days_end'].value_counts()

0    152202
1     62927
Name: DAY_OF_WEEK_days_end, dtype: int64

## Creating New column-> DEP_TIME_BLK as 6 block per day, using CRS_DEP_time with 24 hrs time stamp

In [ ]:
def get_dep_time_blk(dep_time):
    dep_hour = str('%04d' %(dep_time))[:2]
    
    return dep_hour

systematic_sample_selectedcols['DEP_TIMEslot_BLK'] = systematic_sample_selectedcols.DEP_TIME.apply(get_dep_time_blk)
print(systematic_sample_selectedcols[['CRS_DEP_TIME','DEP_TIMEslot_BLK']].head(5))


   CRS_DEP_TIME DEP_TIMEslot_BLK
0          1153               11
1           948               09
2          1010               10
3          1313               13
4           605               05


In [ ]:
systematic_sample_selectedcols[['DEP_TIMEslot_BLK']]=systematic_sample_selectedcols[['DEP_TIMEslot_BLK']].astype(int)

In [ ]:
for dataset in systematic_sample_selectedcols:
    systematic_sample_selectedcols.loc[systematic_sample_selectedcols['DEP_TIMEslot_BLK'] >=0, 'Dep_period'] = 'Midnight'
    systematic_sample_selectedcols.loc[systematic_sample_selectedcols['DEP_TIMEslot_BLK'] >=4,'Dep_period'] ='Early morning'
    systematic_sample_selectedcols.loc[systematic_sample_selectedcols['DEP_TIMEslot_BLK'] >=7,'Dep_period'] ='Morning'
    systematic_sample_selectedcols.loc[systematic_sample_selectedcols['DEP_TIMEslot_BLK'] >=12,'Dep_period'] ='Afternoon'
    systematic_sample_selectedcols.loc[systematic_sample_selectedcols['DEP_TIMEslot_BLK'] >=17,'Dep_period'] ='Evening'
    systematic_sample_selectedcols.loc[systematic_sample_selectedcols['DEP_TIMEslot_BLK'] >=20,'Dep_period'] ='Night'
    systematic_sample_selectedcols.loc[systematic_sample_selectedcols['DEP_TIMEslot_BLK'] ==24, 'Dep_period'] = 'Midnight'


systematic_sample_selectedcols[['DEP_TIMEslot_BLK','Dep_period']].head()


,DEP_TIMEslot_BLK,Dep_period
0,11,Morning
1,9,Morning
2,10,Morning
3,13,Afternoon
4,5,Early morning


In [ ]:
delay_timeslotwise_count = systematic_sample_selectedcols[['Dep_period','DEP_DEL15']].groupby(['Dep_period']).sum('DEP_DEL15').sort_values(['DEP_DEL15'],ascending=False).reset_index()
delay_timeslotwise_count

,Dep_period,DEP_DEL15
0,Afternoon,8049.0
1,Evening,5561.0
2,Morning,5337.0
3,Night,4053.0
4,Early morning,409.0
5,Midnight,270.0


Adding 3 kinds of weightage to categorical columns
1. dep_del count
2. Time slot wise dep delay count
3. Taxi out time Dep delay count

# Column wise weightage 
1. Departure time-  Perodic time slot delay weights 
2. ORIGIN_AIRPORT_ID - Perodic time slot delay weights
3. OP_carrier airline- Perodic time slot delay weights
4. Origin- Perodic time slot delay weights
5. Tail number-Delay weight
6. Days of Week - Delay weight
  * Including Week_days-0 and Week_ends-1 column
7. Days of month - Delay weight
8. Month- Delay weight
9. OP_CARRIER_FL_NUM - Taxi out delay weights

### 1. DEP_TIMEslot_BLK on delay percentage Probablity

In [ ]:
periodcountperdeptime = systematic_sample_selectedcols[['DEP_TIMEslot_BLK','Dep_period']].groupby(['DEP_TIMEslot_BLK','Dep_period']).count().reset_index()
periodcountperdeptime.head()

,DEP_TIMEslot_BLK,Dep_period
0,0,Midnight
1,1,Midnight
2,2,Midnight
3,3,Midnight
4,4,Early morning


In [ ]:
pd.options.display.float_format = '{:.1f}'.format
periodcountperdeptime = systematic_sample_selectedcols.groupby(['Dep_period','DEP_TIMEslot_BLK']).agg({'Dep_period' : ['count'],'DEP_DEL15' : ['sum']}).reset_index()
periodcountperdeptime.columns=['Dep_period','DEP_TIMEslot_BLK','DEP_TIME_timeslot_tot_count','DEP_TIME_timeslotwise_DELAY_COUNT']
periodcountperdeptime.sort_values(['DEP_TIME_timeslotwise_DELAY_COUNT'],ascending=False)
percentage_DEP_TIMEslot= (periodcountperdeptime['DEP_TIME_timeslotwise_DELAY_COUNT']/periodcountperdeptime['DEP_TIME_timeslot_tot_count'])*100
periodcountperdeptime['percentage_DEP_TIMEslot_del']=percentage_DEP_TIMEslot
periodcountperdeptime.sort_values(by='percentage_DEP_TIMEslot_del',ascending=False)
#periodcountperdeptime['DEP_TIME_timeslot_weight'] = pd.qcut(periodcountperdeptime['percentage_DEP_TIMEslot_del'], 24, labels = False,duplicates='drop')
#periodcountperdeptime.sort_values(by='DEP_TIME_timeslot_weight',ascending=False)
print(periodcountperdeptime)
systematic_sample_selectedcols0 = systematic_sample_selectedcols.merge(periodcountperdeptime, left_on=['DEP_TIMEslot_BLK','Dep_period'], right_on=['DEP_TIMEslot_BLK','Dep_period'])
systematic_sample_selectedcols0.drop(columns=['DEP_TIME_timeslotwise_DELAY_COUNT','DEP_TIME_timeslot_tot_count','DEP_TIMEslot_BLK'], inplace=True, axis=1)
systematic_sample_selectedcols0.head(3)

       Dep_period  ...  percentage_DEP_TIMEslot_del
0       Afternoon  ...                         10.6
1       Afternoon  ...                         11.0
2       Afternoon  ...                         11.5
3       Afternoon  ...                         13.1
4       Afternoon  ...                         13.1
5   Early morning  ...                          2.3
6   Early morning  ...                          0.7
7   Early morning  ...                          2.6
8         Evening  ...                         13.2
9         Evening  ...                         14.6
10        Evening  ...                         19.0
11       Midnight  ...                         31.6
12       Midnight  ...                         28.1
13       Midnight  ...                         42.2
14       Midnight  ...                         21.1
15       Midnight  ...                         69.2
16        Morning  ...                          5.0
17        Morning  ...                          5.7
18        Mo

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,ORIGIN,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,TAIL_NUM,AIR_TIME,OP_CARRIER_FL_NUM,DISTANCE,OP_CARRIER_AIRLINE_ID,DEP_DEL15,CRS_DEP_TIME,CRS_ELAPSED_TIME,TAXI_OUT,DEP_TIME,WEEK_OF_MONTH,DAY_OF_WEEK_days_end,Dep_period,percentage_DEP_TIMEslot_del
0,2021,1,17,7,2021-01-17,DFW,11298,11433,N326NB,126.0,1089,986.0,19790,0.0,1153,150.0,28.0,1151.0,3,1,Morning,10.3
1,2021,1,17,7,2021-01-17,MSY,13495,11433,N334NB,118.0,1206,926.0,19790,0.0,1157,147.0,10.0,1155.0,3,1,Morning,10.3
2,2021,1,17,7,2021-01-17,EWR,11618,10397,N991AT,129.0,1255,746.0,19790,0.0,1110,147.0,12.0,1102.0,3,1,Morning,10.3


### 2. ORIGIN - Perodic time slot delay percentage

In [ ]:
periodcountperairport = systematic_sample_selectedcols[['ORIGIN','Dep_period']].groupby(['ORIGIN','Dep_period']).count().reset_index()
periodcountperairport.head(4)

,ORIGIN,Dep_period
0,ABE,Afternoon
1,ABE,Early morning
2,ABE,Evening
3,ABE,Morning


In [ ]:
pd.options.display.float_format = '{:.1f}'.format
periodcountperorg = systematic_sample_selectedcols.groupby(['Dep_period','ORIGIN']).agg({'Dep_period' : ['count'],'DEP_DEL15' : ['sum']}).reset_index()
periodcountperorg.columns=['Dep_period','ORIGIN','ORIGIN_timeslot_tot_count','ORIGIN_timeslotwise_DELAY_COUNT']
periodcountperorg.sort_values(['ORIGIN_timeslotwise_DELAY_COUNT'],ascending=False)
percentage1= (periodcountperorg['ORIGIN_timeslotwise_DELAY_COUNT']/periodcountperorg['ORIGIN_timeslot_tot_count'])*100
periodcountperorg['percentage_Delay_org']=percentage1
periodcountperorg.sort_values(by='percentage_Delay_org',ascending=False)
#periodcountperorg['ORIGIN_timeslot_weight'] = pd.qcut(periodcountperorg['percentage_Delay_org'], 200, labels = False,duplicates='drop')
periodcountperorg.head()
systematic_sample_selectedcols_1 = systematic_sample_selectedcols0.merge(periodcountperorg, left_on=['ORIGIN','Dep_period'], right_on=['ORIGIN','Dep_period'])
systematic_sample_selectedcols_1.drop(columns=['ORIGIN','ORIGIN_timeslot_tot_count','ORIGIN_timeslotwise_DELAY_COUNT'], inplace=True, axis=1)
systematic_sample_selectedcols_1.head(3)

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,TAIL_NUM,AIR_TIME,OP_CARRIER_FL_NUM,DISTANCE,OP_CARRIER_AIRLINE_ID,DEP_DEL15,CRS_DEP_TIME,CRS_ELAPSED_TIME,TAXI_OUT,DEP_TIME,WEEK_OF_MONTH,DAY_OF_WEEK_days_end,Dep_period,percentage_DEP_TIMEslot_del,percentage_Delay_org
0,2021,1,17,7,2021-01-17,11298,11433,N326NB,126.0,1089,986.0,19790,0.0,1153,150.0,28.0,1151.0,3,1,Morning,10.3,10.4
1,2021,1,24,7,2021-01-24,11298,13931,N227NN,140.0,3326,1212.0,20398,1.0,1045,181.0,12.0,1153.0,4,1,Morning,10.3,10.4
2,2021,1,5,2,2021-01-05,11298,15016,N285NN,80.0,3352,550.0,20398,1.0,1019,109.0,12.0,1130.0,2,0,Morning,10.3,10.4


### 3. OP_carrier airline- Perodic time slot delay weights

In [ ]:
periodcountperairLine = systematic_sample_selectedcols[['OP_CARRIER_AIRLINE_ID','Dep_period']].groupby(['OP_CARRIER_AIRLINE_ID','Dep_period']).count().reset_index()
periodcountperairLine.head(5)

,OP_CARRIER_AIRLINE_ID,Dep_period
0,19393,Afternoon
1,19393,Early morning
2,19393,Evening
3,19393,Midnight
4,19393,Morning


In [ ]:
pd.options.display.float_format = '{:.1f}'.format
periodcountperairLine = systematic_sample_selectedcols.groupby(['Dep_period','OP_CARRIER_AIRLINE_ID']).agg({'Dep_period' : ['count'],'DEP_DEL15' : ['sum']}).reset_index()
periodcountperairLine.columns=['Dep_period','OP_CARRIER_AIRLINE_ID','OP_CARRIER_timeslot_TOTcount','OP_CARRIER_timeslotwise_DELAY_COUNT']
periodcountperairLine.sort_values(['OP_CARRIER_timeslotwise_DELAY_COUNT'],ascending=False)
percentage1= (periodcountperairLine['OP_CARRIER_timeslotwise_DELAY_COUNT']/periodcountperairLine['OP_CARRIER_timeslot_TOTcount'])*100
periodcountperairLine['percentage_Delay1']=percentage1
periodcountperairLine.sort_values(by='percentage_Delay1',ascending=False)
#periodcountperairLine['OP_CARRIER_airline_timeslot_weight'] = pd.qcut(periodcountperairLine['percentage_Delay1'], 200, labels = False,duplicates='drop')
systematic_sample_selectedcols_2 = systematic_sample_selectedcols_1.merge(periodcountperairLine, left_on=['OP_CARRIER_AIRLINE_ID','Dep_period'], right_on=['OP_CARRIER_AIRLINE_ID','Dep_period'])
systematic_sample_selectedcols_2.drop(columns=['OP_CARRIER_AIRLINE_ID','OP_CARRIER_timeslotwise_DELAY_COUNT','OP_CARRIER_timeslot_TOTcount'], inplace=True, axis=1)
systematic_sample_selectedcols_2.head()

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,TAIL_NUM,AIR_TIME,OP_CARRIER_FL_NUM,DISTANCE,DEP_DEL15,CRS_DEP_TIME,CRS_ELAPSED_TIME,TAXI_OUT,DEP_TIME,WEEK_OF_MONTH,DAY_OF_WEEK_days_end,Dep_period,percentage_DEP_TIMEslot_del,percentage_Delay_org,percentage_Delay1
0,2021,1,17,7,2021-01-17,11298,11433,N326NB,126.0,1089,986.0,0.0,1153,150.0,28.0,1151.0,3,1,Morning,10.3,10.4,6.3
1,2021,2,24,3,2021-02-24,11298,10397,N305DN,91.0,1244,731.0,1.0,1049,121.0,22.0,1115.0,5,0,Morning,10.3,10.4,6.3
2,2021,2,9,2,2021-02-09,11298,11433,N353NB,118.0,1089,986.0,0.0,1155,150.0,10.0,1147.0,3,0,Morning,10.3,10.4,6.3
3,2021,2,27,6,2021-02-27,11298,11433,N368NB,112.0,1089,986.0,0.0,1155,150.0,21.0,1159.0,5,1,Morning,10.3,10.4,6.3
4,2021,2,13,6,2021-02-13,11298,10397,N329DN,92.0,1244,731.0,1.0,1049,121.0,21.0,1157.0,3,1,Morning,10.3,10.4,6.3


### 5. Tail number-Delay count

In [ ]:
tail_dep_del = systematic_sample_selectedcols[['TAIL_NUM','DEP_DEL15']].groupby('TAIL_NUM').sum().sort_values(by='DEP_DEL15',ascending=False).reset_index()
tail_dep_del.rename({'DEP_DEL15':'TAIL_NUM_DELAY_COUNT'},inplace=True, axis=1)
#tail_dep_del['TAIL_DELAY_weight'] = pd.qcut(tail_dep_del['TAIL_NUM_DELAY_COUNT'], 60, labels = False,duplicates='drop')
print(tail_dep_del)
systematic_sample_selectedcols_3 = systematic_sample_selectedcols_2.merge(tail_dep_del, left_on='TAIL_NUM', right_on='TAIL_NUM')
systematic_sample_selectedcols_3.drop(columns=['TAIL_NUM'], inplace=True, axis=1)
systematic_sample_selectedcols_3.head()

     TAIL_NUM  TAIL_NUM_DELAY_COUNT
0      N957LR                  20.0
1      N952LR                  19.0
2      N425LV                  18.0
3      N430WN                  18.0
4      N7737E                  18.0
...       ...                   ...
5359   N39726                   0.0
5360   N396HA                   0.0
5361   N396DA                   0.0
5362   N14735                   0.0
5363   N773UA                   0.0

[5364 rows x 2 columns]


,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,AIR_TIME,OP_CARRIER_FL_NUM,DISTANCE,DEP_DEL15,CRS_DEP_TIME,CRS_ELAPSED_TIME,TAXI_OUT,DEP_TIME,WEEK_OF_MONTH,DAY_OF_WEEK_days_end,Dep_period,percentage_DEP_TIMEslot_del,percentage_Delay_org,percentage_Delay1,TAIL_NUM_DELAY_COUNT
0,2021,1,17,7,2021-01-17,11298,11433,126.0,1089,986.0,0.0,1153,150.0,28.0,1151.0,3,1,Morning,10.3,10.4,6.3,4.0
1,2021,4,30,5,2021-04-30,11298,13487,118.0,2025,852.0,0.0,905,146.0,14.0,857.0,5,0,Morning,5.7,10.4,6.3,4.0
2,2021,1,18,1,2021-01-18,10397,11624,94.0,2233,646.0,0.0,940,110.0,11.0,934.0,4,0,Morning,8.4,7.4,6.3,4.0
3,2021,5,26,3,2021-05-26,14869,11648,81.0,2126,531.0,0.0,1159,98.0,13.0,1152.0,5,0,Morning,10.3,7.0,6.3,4.0
4,2021,4,6,2,2021-04-06,14869,11884,80.0,2405,546.0,0.0,815,105.0,22.0,810.0,2,0,Morning,5.7,7.0,6.3,4.0


## Including Week_days and Week_ends column 

In [ ]:
#'DAY_OF_WEEK' because delay could depend on traffic in specific days: classify it to weekend or working day
def get_day_category(day_of_week):
    if day_of_week <= 5:
        return 0 #'working day'
    elif day_of_week > 5:
        return 1 #'weekend'
systematic_sample_selectedcols_3['DAY_OF_WEEK_day_end'] = systematic_sample_selectedcols_3['DAY_OF_WEEK']
   
systematic_sample_selectedcols_3.DAY_OF_WEEK_day_end = systematic_sample_selectedcols_3.DAY_OF_WEEK_day_end.apply(get_day_category)
systematic_sample_selectedcols_3.head()

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,AIR_TIME,OP_CARRIER_FL_NUM,DISTANCE,DEP_DEL15,CRS_DEP_TIME,CRS_ELAPSED_TIME,TAXI_OUT,DEP_TIME,WEEK_OF_MONTH,DAY_OF_WEEK_days_end,Dep_period,percentage_DEP_TIMEslot_del,percentage_Delay_org,percentage_Delay1,TAIL_NUM_DELAY_COUNT,DAY_OF_WEEK_day_end
0,2021,1,17,7,2021-01-17,11298,11433,126.0,1089,986.0,0.0,1153,150.0,28.0,1151.0,3,1,Morning,10.3,10.4,6.3,4.0,1
1,2021,4,30,5,2021-04-30,11298,13487,118.0,2025,852.0,0.0,905,146.0,14.0,857.0,5,0,Morning,5.7,10.4,6.3,4.0,0
2,2021,1,18,1,2021-01-18,10397,11624,94.0,2233,646.0,0.0,940,110.0,11.0,934.0,4,0,Morning,8.4,7.4,6.3,4.0,0
3,2021,5,26,3,2021-05-26,14869,11648,81.0,2126,531.0,0.0,1159,98.0,13.0,1152.0,5,0,Morning,10.3,7.0,6.3,4.0,0
4,2021,4,6,2,2021-04-06,14869,11884,80.0,2405,546.0,0.0,815,105.0,22.0,810.0,2,0,Morning,5.7,7.0,6.3,4.0,0


## 6. Days of Week - Delay weight


In [ ]:
cols=['DAY_OF_WEEK','TOTALFLIGHT_WEEKDAY','DEPDELAY_WEEKDAY','WEEK_DEP_DELAY_PERCENTAGE']
lst = []
for week in range(1,8):
  WEEK=week
  weekflighttot=systematic_sample_selectedcols['DAY_OF_WEEK'][systematic_sample_selectedcols.DAY_OF_WEEK==week].count()
  week_all=systematic_sample_selectedcols[systematic_sample_selectedcols['DAY_OF_WEEK']==week]
  weekdepdel=week_all['DEP_DEL15'][week_all['DEP_DEL15']==1].sum()
  percentweekdepdel=(weekdepdel/weekflighttot)*100
  lst.append([WEEK,weekflighttot,weekdepdel,percentweekdepdel])
WEEKWISE_DEP_DELAY_TABLE = pd.DataFrame(lst, columns=cols)
#WEEKWISE_DEP_DELAY_TABLE['WEEK_DELAY_weight'] = pd.qcut(WEEKWISE_DEP_DELAY_TABLE['WEEK_DEP_DELAY_PERCENTAGE'], 7 , labels = False,duplicates='drop')
print(WEEKWISE_DEP_DELAY_TABLE)
systematic_sample_selectedcols_4 = systematic_sample_selectedcols_3.merge(WEEKWISE_DEP_DELAY_TABLE, left_on='DAY_OF_WEEK', right_on='DAY_OF_WEEK')
systematic_sample_selectedcols_4.drop(columns=['TOTALFLIGHT_WEEKDAY','DEPDELAY_WEEKDAY','DAY_OF_WEEK'], inplace=True, axis=1)
systematic_sample_selectedcols_4.head(4)

   DAY_OF_WEEK  ...  WEEK_DEP_DELAY_PERCENTAGE
0            1  ...                       11.6
1            2  ...                        9.2
2            3  ...                        8.9
3            4  ...                       10.6
4            5  ...                       11.3
5            6  ...                       11.6
6            7  ...                       13.2

[7 rows x 4 columns]


,YEAR,MONTH,DAY_OF_MONTH,FL_DATE,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,AIR_TIME,OP_CARRIER_FL_NUM,DISTANCE,DEP_DEL15,CRS_DEP_TIME,CRS_ELAPSED_TIME,TAXI_OUT,DEP_TIME,WEEK_OF_MONTH,DAY_OF_WEEK_days_end,Dep_period,percentage_DEP_TIMEslot_del,percentage_Delay_org,percentage_Delay1,TAIL_NUM_DELAY_COUNT,DAY_OF_WEEK_day_end,WEEK_DEP_DELAY_PERCENTAGE
0,2021,1,17,2021-01-17,11298,11433,126.0,1089,986.0,0.0,1153,150.0,28.0,1151.0,3,1,Morning,10.3,10.4,6.3,4.0,1,13.2
1,2021,2,28,2021-02-28,10721,13487,200.0,963,1124.0,0.0,720,205.0,13.0,715.0,5,1,Morning,5.0,6.0,6.3,4.0,1,13.2
2,2021,5,9,2021-05-09,14869,12954,87.0,833,588.0,1.0,1133,115.0,10.0,1313.0,2,1,Afternoon,11.0,13.8,8.4,4.0,1,13.2
3,2021,1,31,2021-01-31,14107,14747,161.0,1547,1107.0,0.0,1615,193.0,25.0,1608.0,5,1,Afternoon,13.1,13.9,8.4,4.0,1,13.2


## 7. Day of month-Delay weight

In [ ]:
cols=['DAY_OF_MONTH','TOTAL_FLIGHTS','DAY_OF_MONTH_DELAYED_FLIGHTS','%DAYMONTH_DEP_DELAYED']
lst = []
for daymonth in range(1,32):
  MONTHday=daymonth
  monthdayflighttot=systematic_sample_selectedcols['DAY_OF_MONTH'][systematic_sample_selectedcols.DAY_OF_MONTH==daymonth].count()
  monthday_all=systematic_sample_selectedcols[systematic_sample_selectedcols['DAY_OF_MONTH']==daymonth]
  monthdaydepdel=monthday_all['DEP_DEL15'][monthday_all['DEP_DEL15']==1].sum()
  percentmonthdaydepdel=(monthdaydepdel/monthdayflighttot)*100
  lst.append([MONTHday,monthdayflighttot,monthdaydepdel,percentmonthdaydepdel])
MONTHDAYWISE_DEP_DELAY_TABLE = pd.DataFrame(lst, columns=cols)
#MONTHDAYWISE_DEP_DELAY_TABLE['DAY_OF_MONTH_DELAY_weight'] = pd.qcut(MONTHDAYWISE_DEP_DELAY_TABLE['%DAYMONTH_DEP_DELAYED'], 31 , labels = False,duplicates='drop')
print(MONTHDAYWISE_DEP_DELAY_TABLE.head(3))
systematic_sample_selectedcols_5 = systematic_sample_selectedcols_4.merge(MONTHDAYWISE_DEP_DELAY_TABLE, left_on='DAY_OF_MONTH', right_on='DAY_OF_MONTH')
systematic_sample_selectedcols_5.drop(columns=['TOTAL_FLIGHTS','DAY_OF_MONTH_DELAYED_FLIGHTS','DAY_OF_MONTH'], inplace=True, axis=1)
systematic_sample_selectedcols_5.head()

   DAY_OF_MONTH  ...  %DAYMONTH_DEP_DELAYED
0             1  ...                   11.9
1             2  ...                   10.3
2             3  ...                   12.6

[3 rows x 4 columns]


,YEAR,MONTH,FL_DATE,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,AIR_TIME,OP_CARRIER_FL_NUM,DISTANCE,DEP_DEL15,CRS_DEP_TIME,CRS_ELAPSED_TIME,TAXI_OUT,DEP_TIME,WEEK_OF_MONTH,DAY_OF_WEEK_days_end,Dep_period,percentage_DEP_TIMEslot_del,percentage_Delay_org,percentage_Delay1,TAIL_NUM_DELAY_COUNT,DAY_OF_WEEK_day_end,WEEK_DEP_DELAY_PERCENTAGE,%DAYMONTH_DEP_DELAYED
0,2021,1,2021-01-17,11298,11433,126.0,1089,986.0,0.0,1153,150.0,28.0,1151.0,3,1,Morning,10.3,10.4,6.3,4.0,1,13.2,12.2
1,2021,1,2021-01-17,11298,10397,85.0,1244,731.0,0.0,1049,121.0,16.0,1044.0,3,1,Morning,8.2,10.4,6.3,4.0,1,13.2,12.2
2,2021,1,2021-01-17,14679,11433,254.0,799,1956.0,0.0,720,256.0,12.0,704.0,3,1,Morning,5.0,6.2,6.3,5.0,1,13.2,12.2
3,2021,1,2021-01-17,13930,14869,181.0,1305,1250.0,0.0,1700,215.0,32.0,1657.0,3,1,Afternoon,13.1,8.8,8.4,3.0,1,13.2,12.2
4,2021,1,2021-01-17,14869,14771,99.0,1263,599.0,0.0,2105,123.0,11.0,2058.0,3,1,Night,16.9,6.5,14.6,3.0,1,13.2,12.2


## 8. Month - Delay Weight

In [ ]:
cols=['MONTH','TOTAL_FLIGHTS','MONTH_DELAYED_FLIGHTS','%_DEP_DELAYED_month']
lst = []
for monthvar in range(1,6):
  MONTH=monthvar
  monthflighttot=systematic_sample_selectedcols['MONTH'][systematic_sample_selectedcols.MONTH==monthvar].count()
  month_all=systematic_sample_selectedcols[systematic_sample_selectedcols['MONTH']==monthvar]
  monthdepdel=month_all['DEP_DEL15'][month_all['DEP_DEL15']==1].sum()
  percentmonthdepdel=(monthdepdel/monthflighttot)*100
  lst.append([MONTH,monthflighttot,monthdepdel,percentmonthdepdel])
MONTHWISE_DEP_DELAY_TABLE = pd.DataFrame(lst, columns=cols)
#MONTHWISE_DEP_DELAY_TABLE['MONTH_DELAY_weight'] = pd.qcut(MONTHWISE_DEP_DELAY_TABLE['%_DEP_DELAYED'], 5, labels = False)
print(MONTHWISE_DEP_DELAY_TABLE)
systematic_sample_selectedcols_6 = systematic_sample_selectedcols_5.merge(MONTHWISE_DEP_DELAY_TABLE, left_on='MONTH', right_on='MONTH')
systematic_sample_selectedcols_6.drop(columns=['TOTAL_FLIGHTS','MONTH_DELAYED_FLIGHTS','MONTH'], inplace=True, axis=1)
systematic_sample_selectedcols_6.head()

   MONTH  TOTAL_FLIGHTS  MONTH_DELAYED_FLIGHTS  %_DEP_DELAYED_month
0      1          37473                 3198.0                  8.5
1      2          32931                 3821.0                 11.6
2      3          46034                 4646.0                 10.1
3      4          47059                 4998.0                 10.6
4      5          51632                 7016.0                 13.6


,YEAR,FL_DATE,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,AIR_TIME,OP_CARRIER_FL_NUM,DISTANCE,DEP_DEL15,CRS_DEP_TIME,CRS_ELAPSED_TIME,TAXI_OUT,DEP_TIME,WEEK_OF_MONTH,DAY_OF_WEEK_days_end,Dep_period,percentage_DEP_TIMEslot_del,percentage_Delay_org,percentage_Delay1,TAIL_NUM_DELAY_COUNT,DAY_OF_WEEK_day_end,WEEK_DEP_DELAY_PERCENTAGE,%DAYMONTH_DEP_DELAYED,%_DEP_DELAYED_month
0,2021,2021-01-17,11298,11433,126.0,1089,986.0,0.0,1153,150.0,28.0,1151.0,3,1,Morning,10.3,10.4,6.3,4.0,1,13.2,12.2,8.5
1,2021,2021-01-17,11298,10397,85.0,1244,731.0,0.0,1049,121.0,16.0,1044.0,3,1,Morning,8.2,10.4,6.3,4.0,1,13.2,12.2,8.5
2,2021,2021-01-17,14679,11433,254.0,799,1956.0,0.0,720,256.0,12.0,704.0,3,1,Morning,5.0,6.2,6.3,5.0,1,13.2,12.2,8.5
3,2021,2021-01-17,13930,14869,181.0,1305,1250.0,0.0,1700,215.0,32.0,1657.0,3,1,Afternoon,13.1,8.8,8.4,3.0,1,13.2,12.2,8.5
4,2021,2021-01-17,14869,14771,99.0,1263,599.0,0.0,2105,123.0,11.0,2058.0,3,1,Night,16.9,6.5,14.6,3.0,1,13.2,12.2,8.5


## 8. OP_CARRIER_FL_NUM -  Taxi out delay weights

In [ ]:
taxioutmean = systematic_sample_selectedcols[['OP_CARRIER_FL_NUM','TAXI_OUT']].groupby(['OP_CARRIER_FL_NUM']).mean(['TAXI_OUT']).sort_values(by='TAXI_OUT',ascending=False).reset_index()
taxioutmean.rename({'TAXI_OUT':'TAXI_OUT_mean'},inplace=True, axis=1)
taxioutmean['FL_NUM_Taxiout_weight'] = pd.qcut(taxioutmean['TAXI_OUT_mean'], 200, labels = False,duplicates='drop')
print(taxioutmean)
systematic_sample_selectedcols_7 = systematic_sample_selectedcols_6.merge(taxioutmean, left_on='OP_CARRIER_FL_NUM', right_on='OP_CARRIER_FL_NUM')
systematic_sample_selectedcols_7.drop(columns=['TAXI_OUT','TAXI_OUT_mean','OP_CARRIER_FL_NUM','DEP_TIME'], inplace=True, axis=1)
systematic_sample_selectedcols_7.head(3)

      OP_CARRIER_FL_NUM  TAXI_OUT_mean  FL_NUM_Taxiout_weight
0                  9645           74.0                    198
1                  7411           60.0                    198
2                  6822           54.0                    198
3                  7026           48.0                    198
4                  6379           47.0                    198
...                 ...            ...                    ...
6446               6961            6.0                      0
6447               6926            6.0                      0
6448               6666            6.0                      0
6449               6603            6.0                      0
6450               6947            5.0                      0

[6451 rows x 3 columns]


,YEAR,FL_DATE,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,AIR_TIME,DISTANCE,DEP_DEL15,CRS_DEP_TIME,CRS_ELAPSED_TIME,WEEK_OF_MONTH,DAY_OF_WEEK_days_end,Dep_period,percentage_DEP_TIMEslot_del,percentage_Delay_org,percentage_Delay1,TAIL_NUM_DELAY_COUNT,DAY_OF_WEEK_day_end,WEEK_DEP_DELAY_PERCENTAGE,%DAYMONTH_DEP_DELAYED,%_DEP_DELAYED_month,FL_NUM_Taxiout_weight
0,2021,2021-01-17,11298,11433,126.0,986.0,0.0,1153,150.0,3,1,Morning,10.3,10.4,6.3,4.0,1,13.2,12.2,8.5,63
1,2021,2021-01-05,11697,13232,164.0,1167.0,0.0,1925,195.0,2,0,Evening,19.0,19.3,23.5,10.0,0,9.2,8.3,8.5,63
2,2021,2021-05-31,11298,14869,169.0,989.0,0.0,815,177.0,6,0,Morning,5.7,10.4,6.3,9.0,0,11.6,12.5,13.6,63


In [ ]:
systematic_sample_selectedcols8 = systematic_sample_selectedcols_7.drop(columns=['YEAR','FL_DATE'],axis=1)

In [ ]:
systematic_sample_selectedcols8[['DEP_DEL15']]=systematic_sample_selectedcols8[['DEP_DEL15']].astype(int)

In [ ]:
systematic_sample_selectedcols8 = systematic_sample_selectedcols8.astype({col: 'float16' for col in systematic_sample_selectedcols8.select_dtypes('float64').columns})
systematic_sample_selectedcols8 = systematic_sample_selectedcols8.astype({col: 'int16' for col in systematic_sample_selectedcols8.select_dtypes('int64').columns})

In [ ]:
systematic_sample_selectedcols8.dtypes

ORIGIN_AIRPORT_ID                int16
DEST_AIRPORT_ID                  int16
AIR_TIME                       float16
DISTANCE                       float16
DEP_DEL15                        int16
CRS_DEP_TIME                     int16
CRS_ELAPSED_TIME               float16
WEEK_OF_MONTH                    int16
DAY_OF_WEEK_days_end             int16
Dep_period                      object
percentage_DEP_TIMEslot_del    float16
percentage_Delay_org           float16
percentage_Delay1              float16
TAIL_NUM_DELAY_COUNT           float16
DAY_OF_WEEK_day_end              int16
WEEK_DEP_DELAY_PERCENTAGE      float16
%DAYMONTH_DEP_DELAYED          float16
%_DEP_DELAYED_month            float16
FL_NUM_Taxiout_weight            int16
dtype: object

### CORRELATION among features

In [ ]:
corr = systematic_sample_selectedcols8.corr()#Lists all pairs of highly collinear variables
indices = np.where(corr > 0.8)
indices = [(corr.columns[x], corr.columns[y]) for x, y in zip(*indices)
                                        if x != y and x < y]
indices

[('AIR_TIME', 'DISTANCE'),
 ('AIR_TIME', 'CRS_ELAPSED_TIME'),
 ('DISTANCE', 'CRS_ELAPSED_TIME'),
 ('CRS_DEP_TIME', 'percentage_DEP_TIMEslot_del'),
 ('DAY_OF_WEEK_days_end', 'DAY_OF_WEEK_day_end')]

In [ ]:
systematic_samplefinal=systematic_sample_selectedcols8.copy()

In [ ]:
systematic_samplefinal = pd.get_dummies(systematic_samplefinal, drop_first=True)
systematic_samplefinal.columns

Index(['ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID', 'AIR_TIME', 'DISTANCE',
       'DEP_DEL15', 'CRS_DEP_TIME', 'CRS_ELAPSED_TIME', 'WEEK_OF_MONTH',
       'DAY_OF_WEEK_days_end', 'percentage_DEP_TIMEslot_del',
       'percentage_Delay_org', 'percentage_Delay1', 'TAIL_NUM_DELAY_COUNT',
       'DAY_OF_WEEK_day_end', 'WEEK_DEP_DELAY_PERCENTAGE',
       '%DAYMONTH_DEP_DELAYED', '%_DEP_DELAYED_month', 'FL_NUM_Taxiout_weight',
       'Dep_period_Early morning', 'Dep_period_Evening', 'Dep_period_Midnight',
       'Dep_period_Morning', 'Dep_period_Night'],
      dtype='object')

In [ ]:
y = systematic_samplefinal.DEP_DEL15 
 #Drop target and set the dependent values variables 
X = systematic_samplefinal.drop('DEP_DEL15', axis=1)
print(X.shape,y.shape)

(215129, 22) (215129,)


Before deleting features

In [ ]:
from sklearn.linear_model import LogisticRegression
from numpy import mean
model_LOGISTICS = LogisticRegression(class_weight='balanced',solver='saga')
#model_LOGISTICSfit = model_LOGISTICS.fit(principalComponents_train, Y_Train)
pca = PCA(.95)
Robust=RobustScaler()
oversample = SMOTE()
pipeline_LRsmt = make_pipeline(oversample,pca,model_LOGISTICS)
kf = StratifiedKFold(n_splits=5)
y_pred_LR_OSample = cross_val_score(pipeline_LRsmt,X,y, scoring='roc_auc', cv=kf, n_jobs=-1)
print('Mean ROC AUC: %.3f' % mean(y_pred_LR_OSample))

Mean ROC AUC: 0.534


In [ ]:
model_Randomforest_smt= RandomForestClassifier(n_estimators= 100, criterion="gini", class_weight='balanced')  
oversample = SMOTE()
pipeline_RFsmt = make_pipeline(oversample,model_Randomforest_smt)
kf = StratifiedKFold(n_splits=5)
y_pred_RF_OSample = cross_val_score(pipeline_RFsmt,X,y, scoring='roc_auc', cv=kf, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.3f' % mean(y_pred_RF_OSample))

KeyboardInterrupt: ignored

In [ ]:
y_pred_RF_OSample1 = cross_val_predict(pipeline_RFsmt,X, y, cv=kf)
print(confusion_matrix(y, y_pred_RF_OSample1))
print(classification_report(y, y_pred_RF_OSample1))

[[186731   4719]
 [  6492  17187]]
              precision    recall  f1-score   support

           0       0.97      0.98      0.97    191450
           1       0.78      0.73      0.75     23679

    accuracy                           0.95    215129
   macro avg       0.88      0.85      0.86    215129
weighted avg       0.95      0.95      0.95    215129



* Below code, after deleting some features accuracy reduced, so Random forest is well with highly-correlated data that has many interactions. 


After deleting 

In [ ]:
X_new=X.drop(columns=['DAY_OF_WEEK_day_end','DISTANCE','CRS_DEP_TIME'],axis=1)

In [ ]:
model_Randomforest_smt= RandomForestClassifier(n_estimators= 100, criterion="gini", class_weight='balanced')  
oversample = SMOTE()
pipeline_RFsmt = make_pipeline(oversample,model_Randomforest_smt)
kf = StratifiedKFold(n_splits=5)
y_pred_RF_OSample_new = cross_val_score(pipeline_RFsmt,X_new,y, scoring='roc_auc', cv=kf, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.3f' % mean(y_pred_RF_OSample_new))

In [ ]:
X_new8=X.drop(columns=['Dep_period_Early morning', 'Dep_period_Evening', 'Dep_period_Midnight',
       'Dep_period_Morning', 'Dep_period_Night'],axis=1)

In [ ]:
X_new8.columns

Index(['ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID', 'AIR_TIME', 'DISTANCE',
       'CRS_DEP_TIME', 'CRS_ELAPSED_TIME', 'WEEK_OF_MONTH',
       'DAY_OF_WEEK_days_end', 'percentage_DEP_TIMEslot_del',
       'percentage_Delay_org', 'percentage_Delay1', 'TAIL_NUM_DELAY_COUNT',
       'DAY_OF_WEEK_day_end', 'WEEK_DEP_DELAY_PERCENTAGE',
       '%DAYMONTH_DEP_DELAYED', '%_DEP_DELAYED_month',
       'FL_NUM_Taxiout_weight'],
      dtype='object')

In [ ]:
from sklearn.linear_model import LogisticRegression
from numpy import mean
model_LOGISTICS = LogisticRegression(class_weight='balanced',solver='saga')
#model_LOGISTICSfit = model_LOGISTICS.fit(principalComponents_train, Y_Train)
pca = PCA(.95)
Robust=RobustScaler()
oversample = SMOTE()
pipeline_LRsmt = make_pipeline(oversample,pca,model_LOGISTICS)
kf = StratifiedKFold(n_splits=5)
y_pred_LR_OSample8 = cross_val_score(pipeline_LRsmt,X_new8,y, scoring='roc_auc', cv=kf, n_jobs=-1)
print('Mean ROC AUC: %.3f' % mean(y_pred_LR_OSample8))

Mean ROC AUC: 0.534


In [ ]:
model_Randomforest_smt= RandomForestClassifier(n_estimators= 100, criterion="gini", class_weight='balanced')  
oversample = SMOTE()
pipeline_RFsmt = make_pipeline(oversample,model_Randomforest_smt)
kf = StratifiedKFold(n_splits=5)
y_pred_RF_OSample8 = cross_val_score(pipeline_RFsmt,X_new8,y, scoring='roc_auc', cv=kf, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.3f' % mean(y_pred_RF_OSample8))

Mean ROC AUC: 0.864


In [ ]:
y_pred_RF_OSample8_1 = cross_val_predict(pipeline_RFsmt,X_new8, y, cv=kf)
print(confusion_matrix(y, y_pred_RF_OSample8_1))
print(classification_report(y, y_pred_RF_OSample8_1))

[[190394   1056]
 [ 17845   5834]]
              precision    recall  f1-score   support

           0       0.91      0.99      0.95    191450
           1       0.85      0.25      0.38     23679

    accuracy                           0.91    215129
   macro avg       0.88      0.62      0.67    215129
weighted avg       0.91      0.91      0.89    215129



# ARRIVAL DELAY

In [ ]:
#systematic_sample_selectedcols=flightsampledata[['YEAR','MONTH','DAY_OF_MONTH','DAY_OF_WEEK','FL_DATE','ORIGIN','DEST','ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','TAIL_NUM','AIR_TIME','OP_CARRIER_FL_NUM','DISTANCE','OP_CARRIER_AIRLINE_ID','DEP_DEL15','CRS_DEP_TIME','CRS_ARR_TIME','CRS_ELAPSED_TIME','TAXI_OUT','DEP_TIME','TAXI_IN','ARR_TIME','ARR_DEL15']]

NameError: ignored

CREATING NEW COLUMN, ARRIVAL TIME BLOCK

In [ ]:
systematic_sample_selectedcols_ARR['ARR_TIME'].isna().sum()

0

In [ ]:
def get_arr_time_blk(arr_time):
    arr_hour = str('%04d' %(arr_time))[:2]
    
    return arr_hour

systematic_sample_selectedcols['ARR_TIMEslot_BLK'] = systematic_sample_selectedcols.ARR_TIME.apply(get_arr_time_blk)
print(systematic_sample_selectedcols[['ARR_TIME','ARR_TIMEslot_BLK']].head(5))

AttributeError: ignored

In [ ]:
systematic_sample_selectedcols[['ARR_TIMEslot_BLK']]=systematic_sample_selectedcols_ARR[['ARR_TIMEslot_BLK']].astype(int)

In [ ]:
#systematic_sample_selectedcols_ARR[['ARR_TIME','ARR_TIMEslot_BLK']][systematic_sample_selectedcols_ARR['ARR_TIMEslot_BLK']==0]
#systematic_sample_selectedcols_ARR[['ARR_TIME','ARR_TIMEslot_BLK']][systematic_sample_selectedcols_ARR['ARR_TIMEslot_BLK']==24]

For ARRIVAL also create 6 time slot 

In [ ]:
for dataset in systematic_sample_selectedcols_ARR:
    systematic_sample_selectedcols_ARR.loc[systematic_sample_selectedcols_ARR['ARR_TIMEslot_BLK'] >=0, 'arr_period'] = 'Midnight'
    systematic_sample_selectedcols_ARR.loc[systematic_sample_selectedcols_ARR['ARR_TIMEslot_BLK'] >=4,'arr_period'] ='Early morning'
    systematic_sample_selectedcols_ARR.loc[systematic_sample_selectedcols_ARR['ARR_TIMEslot_BLK'] >=7,'arr_period'] ='Morning'
    systematic_sample_selectedcols_ARR.loc[systematic_sample_selectedcols_ARR['ARR_TIMEslot_BLK'] >=12,'arr_period'] ='Afternoon'
    systematic_sample_selectedcols_ARR.loc[systematic_sample_selectedcols_ARR['ARR_TIMEslot_BLK'] >=17,'arr_period'] ='Evening'
    systematic_sample_selectedcols_ARR.loc[systematic_sample_selectedcols_ARR['ARR_TIMEslot_BLK'] >=20,'arr_period'] ='Night'
    systematic_sample_selectedcols_ARR.loc[systematic_sample_selectedcols_ARR['ARR_TIMEslot_BLK'] ==24, 'arr_period'] = 'Midnight'
systematic_sample_selectedcols_ARR[['ARR_TIMEslot_BLK','arr_period']].head()

,ARR_TIMEslot_BLK,arr_period
0,15,Afternoon
1,13,Afternoon
2,12,Afternoon
3,14,Afternoon
4,7,Morning


In [ ]:
systematic_sample_selectedcols_ARR.head()

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,ORIGIN,DEST,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,TAIL_NUM,AIR_TIME,OP_CARRIER_FL_NUM,DISTANCE,OP_CARRIER_AIRLINE_ID,DEP_DEL15,CRS_DEP_TIME,CRS_ARR_TIME,CRS_ELAPSED_TIME,TAXI_OUT,DEP_TIME,TAXI_IN,ARR_TIME,ARR_DEL15,ARR_TIMEslot_BLK,arr_period
0,2021,1,17,7,2021-01-17,DFW,DTW,11298,11433,N326NB,126.0,1089,986.0,19790,0.0,1153,1523,150.0,28.0,1151.0,11.0,1536.0,0.0,15,Afternoon
1,2021,1,17,7,2021-01-17,SLC,DFW,14869,11298,N356NW,122.0,1097,989.0,19790,0.0,948,1333,165.0,31.0,939.0,15.0,1327.0,0.0,13,Afternoon
2,2021,1,17,7,2021-01-17,SEA,SLC,14747,14869,N878DN,80.0,1112,689.0,19790,0.0,1010,1314,124.0,16.0,1006.0,8.0,1250.0,0.0,12,Afternoon
3,2021,1,17,7,2021-01-17,ATL,SDF,10397,14730,N3732J,54.0,1126,321.0,19790,0.0,1313,1426,73.0,14.0,1310.0,6.0,1424.0,0.0,14,Afternoon
4,2021,1,17,7,2021-01-17,DTW,ATL,11433,10397,N819DN,90.0,1140,594.0,19790,0.0,605,823,138.0,26.0,559.0,3.0,758.0,0.0,7,Morning


In [ ]:
systematic_sample_selectedcols_ARR.columns

Index(['YEAR', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE', 'ORIGIN',
       'DEST', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID', 'TAIL_NUM', 'AIR_TIME',
       'OP_CARRIER_FL_NUM', 'DISTANCE', 'OP_CARRIER_AIRLINE_ID', 'DEP_DEL15',
       'CRS_DEP_TIME', 'CRS_ARR_TIME', 'CRS_ELAPSED_TIME', 'TAXI_OUT',
       'DEP_TIME', 'TAXI_IN', 'ARR_TIME', 'ARR_DEL15', 'ARR_TIMEslot_BLK',
       'arr_period'],
      dtype='object')

In [ ]:
systematic_sample_selectedcols.columns

In [ ]:
systematic_samplefinal=systematic_sample_selectedcols[['MONTH','DAY_OF_MONTH','DAY_OF_WEEK','ORIGIN_AIRPORT_ID','DEP_DEL15','DEST_AIRPORT_ID','AIR_TIME','DISTANCE','DEP_TIMEslot_BLK','ARR_TIMEslot_BLK','ARR_DEL15']]

In [ ]:
systematic_samplefinal.dtypes

MONTH                  int64
DAY_OF_MONTH           int64
DAY_OF_WEEK            int64
ORIGIN_AIRPORT_ID      int64
DEP_DEL15            float64
DEST_AIRPORT_ID        int64
AIR_TIME             float64
DISTANCE             float64
DEP_TIMEslot_BLK       int64
ARR_TIMEslot_BLK       int64
ARR_DEL15            float64
dtype: object

In [ ]:
y = systematic_samplefinal.ARR_DEL15 
 #Drop target and set the dependent values variables 
X = systematic_samplefinal.drop('ARR_DEL15', axis=1)
print(X.shape,y.shape)

(215130, 10) (215130,)


In [ ]:
model_Randomforest_smt= RandomForestClassifier(n_estimators= 100, criterion="gini", class_weight='balanced')  
#Rscaler = RobustScaler()
oversample = SMOTE()
pipeline_RFsmt = make_pipeline(oversample,model_Randomforest_smt)

#model_Randomforest_smt = model_Randomforest_smt.fit(X_Train_scaled_smt, y_train_smt)
#y_pred_test_RF = model_Randomforest_smt.predict(X_Test_scaled)
#print('R2 Value RANDOM FOREST:',metrics.r2_score(X_Train_scaled_smt, model_Randomforest_smt.predict(X_Train_scaled_smt)))
kf = StratifiedKFold(n_splits=5)
y_pred_RF_OSample = cross_val_predict(pipeline_RFsmt,X, y, cv=kf)
print(confusion_matrix(y, y_pred_RF_OSample))
print(classification_report(y, y_pred_RF_OSample))

[[183095   6842]
 [  7485  17708]]
              precision    recall  f1-score   support

         0.0       0.96      0.96      0.96    189937
         1.0       0.72      0.70      0.71     25193

    accuracy                           0.93    215130
   macro avg       0.84      0.83      0.84    215130
weighted avg       0.93      0.93      0.93    215130



# ARRIVAL DELAY - APPROACH 1

WITH DEST(ONEHOT) , RAW DATA OF OTHER FIELDS (EXCEPT ARR_TIMEslot_BLK	arr_period(ONEHOT))

In [ ]:
selected_cols_arr1= systematic_sample_selectedcols_ARR[['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK','DEST','AIR_TIME',
       'OP_CARRIER_FL_NUM', 'DISTANCE', 'OP_CARRIER_AIRLINE_ID', 'DEP_DEL15',
       'CRS_DEP_TIME', 'CRS_ARR_TIME', 'CRS_ELAPSED_TIME', 'TAXI_OUT',
       'DEP_TIME', 'TAXI_IN', 'ARR_TIME', 'ARR_TIMEslot_BLK', 'arr_period','ARR_DEL15']]

In [ ]:
selected_cols_arr1.dtypes

MONTH                      int64
DAY_OF_MONTH               int64
DAY_OF_WEEK                int64
DEST                      object
AIR_TIME                 float64
OP_CARRIER_FL_NUM          int64
DISTANCE                 float64
OP_CARRIER_AIRLINE_ID      int64
DEP_DEL15                float64
CRS_DEP_TIME               int64
CRS_ARR_TIME               int64
CRS_ELAPSED_TIME         float64
TAXI_OUT                 float64
DEP_TIME                 float64
TAXI_IN                  float64
ARR_TIME                 float64
ARR_TIMEslot_BLK           int64
arr_period                object
ARR_DEL15                float64
dtype: object

In [ ]:
selected_cols_arr1['DEST'].value_counts()

ATL    12104
DFW    10078
DEN     9202
CLT     8714
ORD     8644
       ...  
MVY        4
LNY        3
MKK        3
ADK        2
OWB        1
Name: DEST, Length: 373, dtype: int64

## Onehot encoding --DESTINATION

In [ ]:
selected_cols_arr1_dummies = pd.get_dummies(selected_cols_arr1, drop_first=True)
selected_cols_arr1_dummies.dtypes

MONTH                         int64
DAY_OF_MONTH                  int64
DAY_OF_WEEK                   int64
AIR_TIME                    float64
OP_CARRIER_FL_NUM             int64
                             ...   
arr_period_Early morning      uint8
arr_period_Evening            uint8
arr_period_Midnight           uint8
arr_period_Morning            uint8
arr_period_Night              uint8
Length: 394, dtype: object

In [ ]:
y_arr1 = selected_cols_arr1_dummies.ARR_DEL15 
#Drop target and set the dependent values variables 
X_arr1 = selected_cols_arr1_dummies.drop('ARR_DEL15', axis=1)
print(X_arr1.shape,y_arr1.shape)

(215129, 393) (215129,)


## LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression
from numpy import mean
model_LOGISTICS = LogisticRegression(class_weight='balanced', solver='saga',penalty='l1')
#model_LOGISTICSfit = model_LOGISTICS.fit(principalComponents_train, Y_Train)
pca = PCA(.95)
Robust=RobustScaler()
oversample = SMOTE()
pipeline_LRsmt_arr1 = make_pipeline(oversample,pca,model_LOGISTICS)
kf = StratifiedKFold(n_splits=5)
y_pred_LR_OSample_arr1 = cross_val_score(pipeline_LRsmt_arr1,X_arr1 , y_arr1, scoring='roc_auc', cv=kf, n_jobs=-1)
print('Mean ROC AUC: %.3f' % mean(y_pred_LR_OSample_arr1))

Mean ROC AUC: 0.575


In [ ]:
model_Randomforest_smt= RandomForestClassifier(n_estimators= 100, criterion="gini", class_weight='balanced')  
oversample = SMOTE()
pipeline_RFsmt_arr1 = make_pipeline(oversample,model_Randomforest_smt)
kf = StratifiedKFold(n_splits=5)
y_pred_RF_OSample_arr1 = cross_val_score(pipeline_RFsmt_arr1, X_arr1, y_arr1, scoring='roc_auc', cv=kf, n_jobs=-1)


In [ ]:
A1_RF_Accuracy=mean(y_pred_RF_OSample_arr1)
print('Mean ROC AUC: %.3f' % A1_RF_Accuracy)

Mean ROC AUC: 0.975


In [ ]:
y_pred_RF_OSample_arr1_pred = cross_val_predict(pipeline_RFsmt_arr1,X_arr1, y_arr1, cv=kf)
print(confusion_matrix(y_arr1, y_pred_RF_OSample_arr1_pred))
print(classification_report(y_arr1, y_pred_RF_OSample_arr1_pred))

[[183994   5880]
 [  5067  20188]]
              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97    189874
         1.0       0.77      0.80      0.79     25255

    accuracy                           0.95    215129
   macro avg       0.87      0.88      0.88    215129
weighted avg       0.95      0.95      0.95    215129



## Lists all pairs of highly correlated variables

In [ ]:
corr = X_arr1.corr()
indices = np.where(corr > 0.8)
indices = [(corr.columns[x], corr.columns[y]) for x, y in zip(*indices)
                                        if x != y and x < y]
indices

[('AIR_TIME', 'DISTANCE'),
 ('AIR_TIME', 'CRS_ELAPSED_TIME'),
 ('DISTANCE', 'CRS_ELAPSED_TIME'),
 ('CRS_DEP_TIME', 'DEP_TIME'),
 ('CRS_ARR_TIME', 'ARR_TIME'),
 ('CRS_ARR_TIME', 'ARR_TIMEslot_BLK'),
 ('ARR_TIME', 'ARR_TIMEslot_BLK')]

Deleting highly correlated columns

In [ ]:
X_arr1_del=X_arr1.drop(columns=['AIR_TIME','DEP_TIME','ARR_TIMEslot_BLK','ARR_TIME'],axis=1)

In [ ]:
X_arr1_del.columns

Index(['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_CARRIER_FL_NUM', 'DISTANCE',
       'OP_CARRIER_AIRLINE_ID', 'DEP_DEL15', 'CRS_DEP_TIME', 'CRS_ARR_TIME',
       'CRS_ELAPSED_TIME',
       ...
       'DEST_XNA', 'DEST_XWA', 'DEST_YAK', 'DEST_YKM', 'DEST_YUM',
       'arr_period_Early morning', 'arr_period_Evening', 'arr_period_Midnight',
       'arr_period_Morning', 'arr_period_Night'],
      dtype='object', length=389)

In [ ]:
from sklearn.linear_model import LogisticRegression
from numpy import mean
model_LOGISTICS = LogisticRegression(class_weight='balanced', solver='saga',penalty='l1')
#model_LOGISTICSfit = model_LOGISTICS.fit(principalComponents_train, Y_Train)
pca = PCA(.95)
Robust=RobustScaler()
oversample = SMOTE()
pipeline_LRsmt_arr1 = make_pipeline(oversample,pca,model_LOGISTICS)
kf = StratifiedKFold(n_splits=5)
y_pred_LR_OSample_arr1_del = cross_val_score(pipeline_LRsmt_arr1,X_arr1_del , y_arr1, scoring='roc_auc', cv=kf, n_jobs=-1)
print('Mean ROC AUC: %.3f' % mean(y_pred_LR_OSample_arr1_del))

Mean ROC AUC: 0.559


Accuracy reduced when eliminating highly correlated columns

# Approach 2

Trials with 'ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID' instead of 'DEST'